In [1]:
# !python check.py

In [2]:
import numpy as np 
import pandas as pd 
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from MyPipeline import CreateCols, FillAgeWithCat, FillServs, FillWithCol
from MyPipeline import FillCNUseLin, FillByFunc, FillVIP, FillCryoSleep, OptionChoose 
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import set_config
set_config(transform_output="pandas")
scoring = 'accuracy'; cv=5
score = {}

In [3]:
train = pd.read_csv('./spaceship-titanic/train.csv')
test = pd.read_csv('./spaceship-titanic/test.csv')
submission = pd.read_csv('./spaceship-titanic/sample_submission.csv')
train.shape, test.shape, submission.shape # check shape

((8693, 14), (4277, 13), (4277, 2))

In [4]:
preprocessing = Pipeline([
    ('create_cols', CreateCols()),            # create colms: g_Id, groupSize, solo, C_deck, C_num, C_side, FName
    ('fill_age_withCat', FillAgeWithCat(func='mean', age_bins=[-1,19,27,38,np.inf])),  # fill Age, create Age_cat
    ('fill_serves', FillServs(func='mean')), 
                        # fill services: RoomService, FoodCourt, ShoppingMall, Spa, VRDeck / create total_services
    ('fill_cabinSide1', FillWithCol('C_side',col=['g_Id'])),
    ('fill_cabinSide2', FillWithCol('FName', col=['g_Id'])),
    ('fill_cabinSide3', FillWithCol('C_side',['FName'])),                                        # C_side mode yet
    ('fill_cabinDeck1', FillWithCol('C_deck',col=['g_Id'])),
    ('fill_cabinDeck2', FillWithCol('C_deck',['HomePlanet','Destination','solo'])),              # C_deck mode yet
    ('fill_UseMode', FillByFunc(['C_side','C_deck','Destination'])),
    ('fill_cabinNum', FillCNUseLin()),                                                            # fill Cabin_num
    ('fill_homeplanet1', FillWithCol('HomePlanet',col=['g_Id'])),
    ('fill_homeplanet2', FillWithCol('HomePlanet',['C_deck'])),                                  # fill HomePlanet
    ('fill_vip', FillVIP()),                                                                            # fill VIP
    ('fill_cryosleep', FillCryoSleep()),                                                           # fill CryoSleep
    ('options', OptionChoose(age_cat=True, solo=True, totalS=False, serv_cat=False, family=False)),   
                                                     # options choose / create dummy columns(onehot) / drop columns
    ('standardScaler', StandardScaler()),                                    # ->>>>>>> standardScaler check please
#     ('poly_features', PolynomialFeatures(degree=2, include_bias=False))     # -> PolynomialFeatures
])

In [5]:
check = preprocessing.fit_transform(train.drop(columns=['Transported']))
check

C:\Users\Playdata\playdata_wj\Project\DA28_pj02_3\MyPipeline.py:122: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  mid_total = X[X['total_servs']<X['total_servs'].quantile((100-self.p)/100)].groupby('VIP').mean()['total_servs'].mean()


,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,C_num,Age_cat_0,Age_cat_1,Age_cat_2,Age_cat_3,...,C_deck_A,C_deck_B,C_deck_C,C_deck_D,C_deck_E,C_deck_F,C_deck_G,C_deck_T,C_side_P,C_side_S
0,-0.337788,-0.284819,-0.287508,-0.274610,-0.266625,-1.172744,-0.574649,-0.601403,-0.587225,1.831128,...,-0.176283,3.149739,-0.309743,-0.243612,-0.336668,-0.709182,-0.651087,-0.02399,1.010059,-1.010059
1,-0.172642,-0.279177,-0.245678,0.213376,-0.227811,-1.172744,-0.574649,1.662779,-0.587225,-0.546111,...,-0.176283,-0.317487,-0.309743,-0.243612,-0.336668,1.410075,-0.651087,-0.02399,-0.990042,0.990042
2,-0.272639,1.957068,-0.287508,5.694116,-0.223400,-1.172744,-0.574649,-0.601403,-0.587225,1.831128,...,5.672688,-0.317487,-0.309743,-0.243612,-0.336668,-0.709182,-0.651087,-0.02399,-0.990042,0.990042
3,-0.337788,0.519527,0.333240,2.684420,-0.096372,-1.172744,-0.574649,-0.601403,1.702925,-0.546111,...,5.672688,-0.317487,-0.309743,-0.243612,-0.336668,-0.709182,-0.651087,-0.02399,-0.990042,0.990042
4,0.121287,-0.240935,-0.034858,0.227598,-0.264861,-1.170792,1.740192,-0.601403,-0.587225,-0.546111,...,-0.176283,-0.317487,-0.309743,-0.243612,-0.336668,1.410075,-0.651087,-0.02399,-0.990042,0.990042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,-0.337788,3.990189,-0.287508,1.185794,-0.201347,-0.981406,-0.574649,-0.601403,-0.587225,1.831128,...,5.672688,-0.317487,-0.309743,-0.243612,-0.336668,-0.709182,-0.651087,-0.02399,1.010059,-1.010059
8689,-0.337788,-0.284819,-0.287508,-0.274610,-0.266625,1.753951,1.740192,-0.601403,-0.587225,-0.546111,...,-0.176283,-0.317487,-0.309743,-0.243612,-0.336668,-0.709182,1.535892,-0.02399,-0.990042,0.990042
8690,-0.337788,-0.284819,2.844675,-0.273722,-0.266625,1.755904,-0.574649,1.662779,-0.587225,-0.546111,...,-0.176283,-0.317487,-0.309743,-0.243612,-0.336668,-0.709182,1.535892,-0.02399,-0.990042,0.990042
8691,-0.337788,0.372826,-0.287508,0.039159,2.587096,0.014334,-0.574649,-0.601403,1.702925,-0.546111,...,-0.176283,-0.317487,-0.309743,-0.243612,2.970286,-0.709182,-0.651087,-0.02399,-0.990042,0.990042


In [6]:
X_train, y_train = train.drop(columns=['Transported']).copy(), train['Transported'].copy()
X_test = test.copy()

full_pipeline = Pipeline([
    ('preprocessing', preprocessing),
    ('RandomForest', RandomForestClassifier())])
full_pipeline.get_params()

{'memory': None,
 'steps': [('preprocessing',
   Pipeline(steps=[('create_cols', CreateCols()),
                   ('fill_age_withCat', FillAgeWithCat()),
                   ('fill_serves', FillServs()),
                   ('fill_cabinSide1', FillWithCol(fill='C_side')),
                   ('fill_cabinSide2', FillWithCol(fill='FName')),
                   ('fill_cabinSide3', FillWithCol(col=['FName'], fill='C_side')),
                   ('fill_cabinDeck1', FillWithCol(fill='C_deck')),
                   ('fill_cabinDeck2',
                    FillWithCol(col=['H...
                                fill='C_deck')),
                   ('fill_UseMode',
                    FillByFunc(fill=['C_side', 'C_deck', 'Destination'])),
                   ('fill_cabinNum', FillCNUseLin()),
                   ('fill_homeplanet1', FillWithCol(fill='HomePlanet')),
                   ('fill_homeplanet2',
                    FillWithCol(col=['C_deck'], fill='HomePlanet')),
                   ('fill_vip', 

In [11]:
param_grid = {
#               'preprocessing__fill_age_withCat__age_bins': [[[-1,19,27,38,np.inf]],[-1,10,20,30,40,50,np.inf]],
              'preprocessing__fill_age_withCat__func': ['mean','median'],
              'preprocessing__fill_serves__func': ['mean','median'],
              'preprocessing__fill_vip__p': [10,5,1],
#               'preprocessing__options__age_cat': [True,False],
              'preprocessing__options__family': [True,False],
              'preprocessing__options__serv_cat': [True,False],
              'preprocessing__options__solo': [True,False],
#               'preprocessing__options__totalS': [True,False],
              'RandomForest__random_state': [42],
#               'RandomForest__criterion': ['gini','entropy','log_loss'],
              'RandomForest__max_depth': [5,10,15,30,50],
              'RandomForest__n_estimators': [1,10,50,100]
              }

grid_search = GridSearchCV(full_pipeline, param_grid, cv=3, scoring=scoring, 
                           return_train_score=True, n_jobs=-1, verbose=4)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 1920 candidates, totalling 5760 fits


KeyboardInterrupt: 

In [ ]:
grid_search.best_params_, grid_search.best_score_

In [ ]:
sorted(list(zip(
    grid_search.best_estimator_[-1].feature_importances,
    grid_search.best_estimator_[:-1].get_feature_names_out(list(X_train.columns)))),reverse=True)

In [ ]:
# pre_train = grid_search.best_estimator_[:-1].transform(X_train)
# pre_test = grid_search.best_estimator_[:-1].transform(X_test)
# best_est = grid_search.best_estimator_[-1]

In [ ]:
pred = grid_search.predict(X_test)
submission['Transported'] = pred
submission.head()

In [ ]:
ver = '1'
submission.to_csv('./submission_SpaceshipTitanic_pipeline_{}.csv'.format(ver), index=False)